In [ ]:
import mediapipe as mp
import cv2
import numpy as np
import os
import time
import uuid

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands


<img src="https://i.imgur.com/qpRACer.png "/>

In [ ]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img, 1)
        img.flags.writeable = False
        res = hands.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if res.multi_hand_landmarks:
            for num , hand in enumerate(res.multi_hand_landmarks):
                mp_drawing.draw_landmarks(img, hand, mp_hands.HAND_CONNECTIONS)
            for hand in res.multi_hand_landmarks:
                coord = str(round(hand.landmark[8].x*640,2))+","+str(round(hand.landmark[8].y*480,2))
        
                cv2.putText(img, coord, tuple(np.multiply(np.array([hand.landmark[8].x,hand.landmark[8].y]), [640,480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 1, (121, 93, 60), 1, cv2.LINE_AA)


        cv2.imshow("Hand Tracking", img)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
for idx,classification in enumerate(res.multi_handedness):
    print(classification.classification[0].index)

In [ ]:
len(res.multi_handedness)

In [ ]:
# def get_label(index,hand,res):
#     output = None
#     if len(res.multi_handedness)==1:
#         label = res.multi_handedness[0].classification[0].label
#         score = res.multi_handedness[0].classification[0].score
#         text = '{} {}'.format(label,round(score,2))

#         coords = tuple(np.multiply(
#             np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x,hand.landmark[mp_hands.HandLandmark.WRIST].y)),
#             [640,480]).astype(int))
#         output = text, coords
#         print(label,coords)
#         return output
#     elif len(res.multi_handedness)>1:
#         for idx, classification in enumerate(res.multi_handedness):
#             if classification.classification[0].index == index:
#                 label = classification.classification[0].label
#                 score = classification.classification[0].score
#                 text = '{} {}'.format(label,round(score,2))
#                 coords = tuple(np.multiply(
#                     np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x,hand.landmark[mp_hands.HandLandmark.WRIST].y)),
#                     [640,480]).astype(int))
#                 print(label,coords)
#                 output = text, coords
#         return output
#     return output
              

In [ ]:
def get_label(index, hand, results):
    output = None

    if index == 0:
        label = results.multi_handedness[0].classification[0].label
        score = results.multi_handedness[0].classification[0].score
        coords = tuple(np.multiply(
                        np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x, hand.landmark[mp_hands.HandLandmark.WRIST].y)),
                        [640,480]).astype(int))

        output = '{}{}'.format(label,round(score,2)), coords
        return output
    
    if index == 1:
        label = results.multi_handedness[1].classification[0].label
        coords = tuple(np.multiply(
                        np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x, hand.landmark[mp_hands.HandLandmark.WRIST].y)),
                        [640,480]).astype(int))
        score = results.multi_handedness[1].classification[0].score
        

        output = '{}{}'.format(label,round(score,2)), coords
        return output

In [ ]:

# get_label(0,hand,res)

In [ ]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img, 1)
        img.flags.writeable = False
        res = hands.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if res.multi_hand_landmarks:
            for num , hand in enumerate(res.multi_hand_landmarks):
                mp_drawing.draw_landmarks(img, hand, mp_hands.HAND_CONNECTIONS)

                
                
                if get_label(num,hand,res):
                    text, coords = get_label(num,hand,res)
                    cv2.putText(img, text, coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (121, 93, 60), 1, cv2.LINE_AA)




        cv2.imshow("Hand Tracking", img)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

        
cap.release()
cv2.destroyAllWindows()

In [ ]:

import matplotlib.pyplot as plt

In [ ]:
top_joint_list = [[4,3,2],[8,7,6],[12,11,10],[16,15,14],[20,19,18]]
bottom_joint_list = [[2,1,0],[6,5,0],[10,9,0],[14,13,0],[18,17,0]]

In [ ]:
def draw_finger_angles(img,res,joint_list,color):
    angle_list = []
    if res.multi_hand_landmarks:
        for hand in res.multi_hand_landmarks:
            for joint in joint_list:
                coord1 = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y])
                coord2 = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y])
                coord3 = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y])
                radians = np.arctan2(coord3[1] - coord2[1], coord3[0] - coord2[0]) - np.arctan2(coord1[1] - coord2[1], coord1[0] - coord2[0])
                angle = np.abs(radians*180.0/np.pi)
                # if angle > 180.0:
                #     angle = 360-angle
                angle_list.append(angle)
                cv2.putText(img, str(round(angle,2)), tuple(np.multiply(coord2, [640,480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
    return img,angle_list

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
img,angle_list=draw_finger_angles(img,res,top_joint_list,(0,0,0))
print(angle_list)

In [ ]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
len(res.multi_hand_landmarks)

In [ ]:
print(angle_list)

In [ ]:
img,bottom_angle_list=draw_finger_angles(img,res,bottom_joint_list,(255,255,255))

In [ ]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
print(bottom_angle_list)

In [ ]:
top_angles=[]
bottom_angles=[]

In [ ]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img, 1)
        img.flags.writeable = False
        res = hands.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if res.multi_hand_landmarks:
            for num , hand in enumerate(res.multi_hand_landmarks):
                mp_drawing.draw_landmarks(img, hand, mp_hands.HAND_CONNECTIONS)

                
                
                if get_label(num,hand,res):
                    text, coords = get_label(num,hand,res)
                    cv2.putText(img, text, coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (121, 93, 60), 1, cv2.LINE_AA)


            img,top_angle_list = draw_finger_angles(img,res,bottom_joint_list,(0,0,0))
            img,bottom_angle_list = draw_finger_angles(img,res,top_joint_list,(255,255,255))
            top_angles.append(top_angle_list)
            bottom_angles.append(bottom_angle_list)

        cv2.imshow("Hand Tracking", img)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

        
cap.release()
cv2.destroyAllWindows()

In [ ]:
print(top_angle_list)

In [ ]:
print(top_angle_list,bottom_angle_list)

In [ ]:
def calc_max_min_angles(angle_list):
    max_angle=[0,0,0,0,0]
    min_angle=[1000,1000,1000,1000,1000]
    for angle in angle_list:
        for i in range(5):
            if angle[i]>max_angle[i]:
                max_angle[i]=angle[i]
            if angle[i]<min_angle[i]:
                min_angle[i]=angle[i]
    return max_angle,min_angle
     

In [ ]:
max_top_angles,min_top_angles=calc_max_min_angles(top_angles)

In [ ]:
max_bottom_angles,min_bottom_angles = calc_max_min_angles(bottom_angles)

In [ ]:
print(max_bottom_angles,max_top_angles)
    

In [ ]:
print(min_bottom_angles,min_top_angles)

In [ ]:
def pred(max_bottom_angles,max_top_angles,min_bottom_angles,min_top_angles,top_angles,bottom_angles):
    score = []
    for i in range(len(max_bottom_angles)):
        mid = (min_top_angles[i]+max_top_angles[i])/2
        score.append( 1-((mid-top_angles[i])**2)/((min_top_angles[i]-max_top_angles[i])**2))
    score = np.array(score)
    s1 = np.sum(score)/len(score)
    score = []
    for i in range(len(max_bottom_angles)):
        mid = (min_bottom_angles[i]+max_bottom_angles[i])/2
        score.append( 1-((mid-bottom_angles[i])**2)/((min_bottom_angles[i]-max_bottom_angles[i])**2))
    score = np.array(score)
    s2 = np.sum(score)/len(score)
    s = (s1+s2)/2
    if(s>0.5):
        return "thumbs up",s
    return "idk",s

In [ ]:
test1 = [168.33527221839458, 109.61749407565415, 128.78987218954217, 155.06870223021772, 164.3271516176781]
test2 = [160.2158443845158, 140.408322256868, 155.065872636256, 165.05621459895184, 175.2198450599878]
pred(max_bottom_angles,max_top_angles,min_bottom_angles,min_top_angles,test1,test2)

In [ ]:
cap = cv2.VideoCapture(0)
_,frame = cap.read()
h ,w , c = frame.shape
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img, 1)
        img.flags.writeable = False
        res = hands.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if res.multi_hand_landmarks:
            for num , hand in enumerate(res.multi_hand_landmarks):
                mp_drawing.draw_landmarks(img, hand, mp_hands.HAND_CONNECTIONS)
                x_max = 0
                y_max = 0
                x_min = w
                y_min = h
                for lm in hand.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    if x > x_max:
                        x_max = x
                    if x < x_min:
                        x_min = x
                    if y > y_max:
                        y_max = y
                    if y < y_min:
                        y_min = y

                cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                
                
                if get_label(num,hand,res):
                    text, coords = get_label(num,hand,res)
                    cv2.putText(img, text, coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (121, 93, 60), 1, cv2.LINE_AA)

                

                img,top_angle_list = draw_finger_angles(img,res,bottom_joint_list,(0,0,0))
                img,bottom_angle_list = draw_finger_angles(img,res,top_joint_list,(255,255,255))
                text,score = pred(max_bottom_angles,max_top_angles,min_bottom_angles,min_top_angles,top_angle_list,bottom_angle_list)
                cv2.putText(img, '{}{}'.format(text,round(score,2)), (x_max, y_min), cv2.FONT_HERSHEY_SIMPLEX, 1, (121, 93, 60), 1, cv2.LINE_AA)
            
        cv2.imshow("Hand Tracking", img)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

        
cap.release()
cv2.destroyAllWindows()